In [1]:
pip install split-folders

In [2]:
import splitfolders

In [3]:
# Menggunakan Google Colab untuk mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
input_folder = '/content/drive/MyDrive/dataset'
output_folder = '/content/drive/MyDrive/capstone'

In [7]:
splitfolders.ratio(input_folder, output=output_folder, seed=38, ratio=(.7, .3))

Copying files: 2145 files [05:53,  6.07 files/s]


In [9]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf


# Menentukan direktori data
data_dir = "/content/drive/MyDrive/capstone"
train_data_dir = os.path.join(data_dir, 'train')
validation_data_dir = os.path.join(data_dir, 'val')

# Mendapatkan daftar kategori (kelas) makanan
train_directories = sorted(os.listdir(train_data_dir), key=str.lower)
validation_directories = sorted(os.listdir(validation_data_dir), key=str.lower)

print(train_directories)
print(validation_directories)

# Augmentasi Data
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

# Menentukan jumlah kelas
num_classes = len(train_directories)

# Transfer Learning dengan MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Membekukan lapisan di base model
for layer in base_model.layers:
    layer.trainable = False

# Menambahkan lapisan klasifikasi di atas base model
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))
# Mengompilasi model
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callback untuk early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Melatih model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping]
)


['Baso', 'Dadar Gulung', 'Gudeg', 'Kerak Telor', 'Nasi Goreng', 'Nasi Kuning', 'Nasi Pecel', 'Peuyeum', 'Putu Ayu', 'Rawon', 'Serabi', 'Soto Lamongan', 'Tahu Sumedang', 'Telur Balado']
['Baso', 'Dadar Gulung', 'Gudeg', 'Kerak Telor', 'Nasi Goreng', 'Nasi Kuning', 'Nasi Pecel', 'Peuyeum', 'Putu Ayu', 'Rawon', 'Serabi', 'Soto Lamongan', 'Tahu Sumedang', 'Telur Balado']
Found 1493 images belonging to 14 classes.
Found 652 images belonging to 14 classes.
9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/100
46/46 [==============================] - 53s 1s/step - loss: 2.4790 - accuracy: 0.2724 - val_loss: 1.6917 - val_accuracy: 0.5188
Epoch 2/100
46/46 [==============================] - 47s 1s/step - loss: 1.8221 - accuracy: 0.4559 - val_loss: 1.3317 - val_accuracy: 0.6156
Epoch 3/100
46/46 [==============================] - 41s 890ms/step - loss: 1.5321 - accuracy: 0.5311 - val_loss: 1.0915 - val_accuracy: 0.6859
Epoch 4/100
46/46 [==============================] - 38s

In [10]:
# Evaluate the model
loss, accuracy = model.evaluate(train_generator)
print(f"Accuracy: {accuracy}")

val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy}")

# Save the entire model
model_save_path = 'my_model.h5'
model.save(model_save_path)

47/47 [==============================] - 29s 607ms/step - loss: 0.1312 - accuracy: 0.9658
Accuracy: 0.9658405780792236
21/21 [==============================] - 13s 593ms/step - loss: 0.2886 - accuracy: 0.9003
Validation Accuracy: 0.900306761264801


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
tflite_model_path = 'my_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

In [12]:
# Buat file label.txt
label_content = """
Bakso
Dadar Gulung
Gudeg
Kerak Telor
Nasi Goreng
Nasi Kuning
Nasi Pecel
Peuyeum
Putu Ayu
Rawon
Serabi
Soto Lamongan
Tahu Sumedang
Telur Balado
"""
with open('label.txt', 'w') as f:
    f.write(label_content.strip())

In [14]:
# Pastikan Anda menginstal tflite-support
!pip install tflite-support

In [35]:
from tflite_support.metadata_writers import image_classifier
from tflite_support.metadata_writers import writer_utils

# Path to the model file
model_path = "my_model.tflite"

# Path to the label file
label_file_path = "label.txt"

# Path to save the new model with metadata
model_with_metadata_path = "/content/drive/MyDrive/model_with_metadata.tflite"

# Memuat file model
model_content = writer_utils.load_file(model_path)

# Membuat writer untuk menambahkan metadata
writer = image_classifier.MetadataWriter.create_for_inference(
    model_content,
    input_norm_mean=[127.5],
    input_norm_std=[127.5],
    label_file_paths=[label_file_path]
)

# Menyimpan model baru dengan metadata
writer_utils.save_file(writer.populate(), model_with_metadata_path)

# Mengunduh file model baru
from google.colab import files
files.download(model_with_metadata_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path='model_with_metadata (3).tflite')
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function to preprocess the image
def preprocess_image(image_path):
    img = Image.open(image_path).resize((128, 128))  # Adjust the size according to your model
    img = np.array(img).astype(np.float32)
    img = (img / 127.5) - 1  # Normalize image to [-1, 1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Function to postprocess the output
def postprocess_output(output_data):
    probabilities = tf.nn.softmax(output_data[0])  # Apply softmax to get probabilities
    class_index = np.argmax(probabilities)  # Get the index of the highest probability
    return class_index, probabilities[class_index]

# Load label file
label_file_path = 'label.txt'
with open(label_file_path, 'r') as f:
    labels = f.read().splitlines()

# Path to the test image
test_image_path = 'Screenshot 2024-06-16 200719.jpg'

# Preprocess the image
input_data = preprocess_image(test_image_path)

# Set the input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Postprocess the output to get the class index and its probability
class_index, class_probability = postprocess_output(output_data)
food_name = labels[class_index]


print(f'The model predicts this image is: {food_name}')


The model predicts this image is: Rawon
